In [5]:
%%capture
%run ./load_concatenated_text.ipynb


In [6]:
import spacy
from spellchecker import SpellChecker

nlp = spacy.load('en_core_web_sm')

spell = SpellChecker()

#Here's a list of words that aren't in the spell checker but that we should probably preserve
#We'll probably need to iterate on this list
spell.word_frequency.load_words([
    'Arlington'
    , 'Glebe'
    , 'Ballston'
    , 'Rosslyn'
    , 'Pershing'
    , 'Rockville'
    , 'MD'
    , 'VA'
    , 'Maryland'
    , 'Virginia'
    , 'Bluemont'
    , 'Wilson'
    ])
# Function to spell-check text. We'll use apply() to run this function on every record
def spell_check(text):
    doc = nlp(text)  # Process the text with spaCy
    corrected_words = []
    
    # Find misspelled words
    misspelled = spell.unknown([token.text for token in doc if not token.is_punct and not token.is_stop])

    for token in doc:
        if not token.is_punct and not token.is_stop and len(token.text.strip()) > 0:  # Exclude punctuation and stop words
            word = token.text.strip()
            if word.lower() in misspelled:
                correction = spell.correction(word)
                if (correction is not None) and (correction.lower() != word.lower()):
                    corrected_words.append(correction)
                    #Uncomment this line to review the list of words that are correcting
                    #print(f"Correcting {word} => {correction}")
                else:
                   corrected_words.append(word.lower())
            else:
                corrected_words.append(word)  # Preserve correct words

    if len(corrected_words)>0:
        return " ".join(corrected_words)
    else: 
        return ""
# Apply the spell-checking function to the DataFrame
df_events['spell_checked_text'] = df_events['concatenated_text'].apply(spell_check)
#Take a look at the result
df_events

,id,concatenated_text,ignore,source,themes,language,demographics,spell_checked_text
1,K1,Housing available at all income levels w/ \now...,NaN,Kickoff,"affordable housing, home ownership, home value...",NaN,NaN,Housing available income levels we ownership a...
2,K2,because in the mid 2020s Arlington focused on ...,NaN,Kickoff,"Noise pollution, air quality, public transit",NaN,NaN,mid 2020s Arlington focused creating great pla...
3,K3,Arlington's investment in next gen smart growt...,NaN,Kickoff,"Aging in place, housing stock, transit",NaN,NaN,Arlington investment get smart growth paid fou...
4,K4,More multicultural and diversity friendly envi...,NaN,Kickoff,"Diversity, cultural exchange, climate resilien...",NaN,NaN,multicultural diversity friendly environment F...
5,K5,Increased regional cooperation. There are hard...,NaN,Kickoff,"regional cooperation, car-free mobility",NaN,NaN,Increased regional cooperation hardly cars str...
...,...,...,...,...,...,...,...,...
454,DO35,Acceptance. Love. Peace.,NaN,Shirlington Library,Community,NaN,NaN,Acceptance Love Peace
455,DO36,Still lots of trees and open spaces. Also loca...,NaN,Shirlington Library,Nature and small businesses,NaN,NaN,lots trees open spaces locally owned stores
456,DO37,"There are a lot of parks, libraries, and schools.",DO 38-39,Shirlington Library,3rd places and education,NaN,NaN,lot parks libraries schools
457,DO38,Coz its pretty,NaN,Shirlington Library,Aesthetic,NaN,NaN,Coz pretty
